In [5]:
# Implementing libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
os.getcwd()

'/content'

In [8]:
os.chdir("/content/drive/MyDrive/Netflix-Dataset/")

In [10]:
dataset_path = os.getcwd()

In [12]:
dataset_path

'/content/drive/MyDrive/Netflix-Dataset'

In [13]:
# Read the dataset

df = pd.read_csv(os.path.join(dataset_path, "Input_Dataset.csv"), header = None, names = ['Cust_Id', 'Rating', 'Date'], usecols = [0,1])
df['Rating'] = df['Rating'].astype(float)